# Using a transferred DCNN. 

*   Take a DCNN trained on image classification
*   Adapt to spectrogram classification
  




Inspired by "Micro-Doppler Based Classification of Human Aquatic Activities via Transfer Learning of Convolutional Neural Networks"

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

Needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [2]:
import os
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif os.getcwd() == 'C:\\Users\\macka\\Google Drive\\Level-4-Project\\notebooks':
    BASE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/"
    
else:
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/cifar_initialised/'
MODEL_PATH = BASE_PATH + 'models/transferred_DCNN/'
FIGURE_PATH = BASE_PATH + 'reports/figures/transferred_DCNN/'
REPORT_PATH = BASE_PATH + 'reports/transferred_DCNN/'
    
from src.models.original_models import cnn_various_depths
from src.data import load_data
from src.visualization import visualize

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


Import remaining packages

In [0]:
import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [0]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

## Experiment Setup and Parameter Definition

In [0]:
load_models = False  # if True, load saved models instead of retraining

batch_size = 100
nb_classes = 7
nb_epoch = 3
nb_train_samples = 35595

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB

In [0]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = load_data.load_data(DATA_PATH, nb_train_samples)
 

target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']

Stack three times to fit VGG16 (RGB) treated as grayscale

In [7]:
print(X_train.shape)
X_train = np.hstack((X_train, X_train, X_train))
X_test = np.hstack((X_test, X_test, X_test))
print(X_train.shape)

(35595, 1, 75, 75)
(35595, 3, 75, 75)


In [0]:
def make_model(img_rows, img_cols, nb_classes):
    model = VGG16(weights = "imagenet", include_top=False, input_shape=(3, img_rows, img_cols))
    x = model.output
    x = Flatten()(x)
    # x = Dense(512, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    # x = Dense(512, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(nb_classes, activation="softmax")(x)
    model = Model(input = model.input, output = predictions)
    sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])
    return model

In [9]:
print(X_train.shape, y_train.shape)

(35595, 3, 75, 75) (35595, 1)


## Training and Evaluating Models

In [0]:
def train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, target_names, nb_classes):
    # convert class vectors to binary class matrices
    labels_train = np_utils.to_categorical(labels_train, nb_classes)
    labels_test = np_utils.to_categorical(labels_test, nb_classes)
    data_train = data_train.astype('float32')
    data_test = data_test.astype('float32')
    data_train /= 255
    data_test /= 255
    history = model.fit(
        data_train,
        labels_train,
        batch_size=batch_size,
        epochs=nb_epoch,
        validation_data=(data_test, labels_test),
        shuffle=True, 
        verbose=1)
    y_pred = model.predict(data_test)
    y_pred = np.argmax(y_pred,axis=1)
    classification_rep = classification_report(np.argmax(labels_test,axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(labels_test,axis=1), y_pred)     
    print(classification_rep)

In [16]:
n_folds = 3
skf = StratifiedKFold(n_splits=n_folds, random_state=None, shuffle=True)
i = 0
for train_index, test_index in skf.split(X_train, y_train):
    print ("Running Fold", i+1, "/", n_folds)
    model = None # Clearing the NN.
    model = make_model(img_rows, img_cols, nb_classes)
    print(train_index, test_index)
    train_and_evaluate_model(model, X_train[train_index], y_train[train_index], X_train[test_index], y_train[test_index], target_names, nb_classes)
    i+=1

Running Fold 1 / 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


[    1     3     6 ... 35589 35590 35591] [    0     2     4 ... 35592 35593 35594]
Train on 23730 samples, validate on 11865 samples
Epoch 1/3
23730/23730 [==============================] - 109s 5ms/step - loss: 1.1796 - acc: 0.5623 - val_loss: 0.1487 - val_acc: 0.9469
Epoch 2/3
23730/23730 [==============================] - 108s 5ms/step - loss: 0.1027 - acc: 0.9673 - val_loss: 0.0336 - val_acc: 0.9883
Epoch 3/3
23730/23730 [==============================] - 108s 5ms/step - loss: 0.0410 - acc: 0.9882 - val_loss: 0.0145 - val_acc: 0.9956
                  precision    recall  f1-score   support

ArmFasterTowards       0.99      1.00      0.99      1695
ArmSlowerTowards       0.99      0.99      0.99      1695
     CirclingArm       0.99      1.00      0.99      1695
        Clapping       1.00      1.00      1.00      1695
       PickingUp       1.00      0.99      0.99      1695
         Sitting       1.00      0.99      0.99      1695
         Walking       1.00      1.00      1.00 

## Analysis and Saving of Results

In [0]:
save_graph = False
save_models = False
save_reports = False

### Plot and Save graph

In [15]:
if not load_models:
    visualize.plot_train_test_acc(history)


NameError: ignored

### Save Models

In [0]:
if not load_models and save_models:
    model.save(MODEL_PATH + name + '.h5')

### Save Classification Report and Confusion Matricies

In [0]:
if save_reports:
    file = open(REPORT_PATH + 'Classification_and_Confusion.txt', 'w') 

    file.write("--------------------------------------------------\n") 
    file.write("Model Name: " + name + "\n") 
    file.write("Classification Report:\n") 
    file.write(classification_rep)
    file.write("Confusion Matrix:\n") 
    file.write(np.array2string(confusion) + "\n")

    file.close() 